In [1]:
import pandas as pd
from datetime import date, datetime
import sys
# sys.path.insert(1, 'C:/Users/maperezr/proyectos/conc')
# from dics import pos_loc
pd.options.display.float_format ='${:,.0f}'.format
pd.options.display.max_columns = 500
dt = datetime.now().strftime("%y%m%d")

Cargar archivos

In [2]:
ventas = pd.read_excel('input\consolidado_ventas_21.xlsx', dtype = str)
nc = pd.read_csv('input/consolidado_nc.csv', dtype = str)

In [3]:
planta = pd.read_csv('input/221012_planta.csv', sep=';', dtype = str)

In [4]:
temporal = pd.read_csv('input/221012_activos.csv', sep=';', dtype = str)

In [5]:
empleados = pd.read_csv('input/221005_colaboradores.csv', encoding = 'utf-16', dtype= str)

In [6]:
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('ene', '01')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('feb', '02')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('mar', '03')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('abr', '04')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('may', '05')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('jun', '06')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('jul', '07')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('ago', '08')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('sep', '09')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('oct', '10')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('nov', '11')
nc.Dcompra_nvo = nc.Dcompra_nvo.str.replace('dic', '12')
nc.Dcompra_nvo = pd.to_datetime(nc.Dcompra_nvo, format='%d-%m-%Y') 
nc['Mventa_nc'] = pd.to_numeric(nc['Mventa_nc'])
nc['Mventa_nc'] = abs(nc['Mventa_nc'])
ventas['Venta en $'] = pd.to_numeric(ventas['Venta en $'])


In [7]:
nc['Cvendedor'].fillna(0, inplace=True)

In [8]:
pos_cyc= {'6': ['3'], 
        '35': ['13'], 
        '50': ['3'], 
        '53': ['15'], 
        '60': ['46'], 
        '93': ['21'], 
        '96': ['22'], 
        '101': ['28'], 
        '123': ['47'], 
        '131': ['13'], 
        '183': ['38']}

pos_what = {'13': ['42'],
            '19': ['16'],
            '25': ['40'],
            '35': ['33'],
            '43': ['47'],
            '60': ['5'],
            '72': ['2'],
            '85': ['18'],
            '96': ['17'],
            '98': ['4'],
            '101': ['18'],
            '108': ['33']}
            
pos_sac = {'6': ['95'],
        '13': ['95'],
        '19': ['95'],
        '25': ['96'],
        '30': ['96'],
        '35': ['96'],
        '36': ['95'],
        '38': ['96'],
        '43': ['96'],
        '45': ['95'],
        '50': ['3'],
        '53': ['95'],
        '56': ['95'],
        '60': ['5','96'],
        '72': ['95'],
        '82': ['95'],
        '85': ['96', '97'],
        '93': ['95'],
        '96': ['96'],
        '98': ['96'],
        '101': ['95', '96'],
        '108': ['97'],
        '123': ['96'],
        '131': ['96'],
        '138': ['96'],
        '183': ['95', '96']}

In [9]:
for i in pos_sac:
    nc.loc[(nc['Local_creacion'] == i) & (nc['Nterminal_nvo'].isin(pos_sac[i])), 'ind_pos'] = 'Pos SAC'
    nc.loc[nc['ind_pos'].isna(), 'ind_pos'] = 'Pos Tienda'


In [10]:
planta['NOMBRE COMPLETO'] = planta['Nombre Empleado'] + ' ' +  planta['Apellido Paterno'] + ' ' + planta['Apellido Materno'] 

In [11]:
empleados.rename(columns={'Unnamed: 2' :'Nombre'}, inplace=True)
empleados.drop_duplicates(['Nombre','Número Vendedor'], inplace= True)

In [12]:
empleados = empleados.loc[empleados['Número Vendedor'] != '0']

In [13]:
empleados['Nombre'] = empleados['Nombre'].apply(lambda x : ' '.join(str(x).split()))

In [14]:
nc_emp = nc.merge(planta[['Codigo Empleado', 'NOMBRE COMPLETO','Cargo', 'Sucursales' ]], how= 'left', left_on = 'Cvendedor', right_on = 'Codigo Empleado')
nc_emp = nc_emp.merge(empleados, how='left', left_on = 'Cvendedor', right_on = 'Número Vendedor')
nc_emp.loc[(nc_emp['NOMBRE COMPLETO'].isna()) & (nc_emp['Nombre'].notna()), 'NOMBRE COMPLETO'] = nc_emp.loc[(nc_emp['NOMBRE COMPLETO'].isna()) & (nc_emp['Nombre'].notna()), 'Nombre'].values

In [15]:
# nc_mes = nc_emp.loc[(nc_emp['Dcompra_nvo'] >= '2022-09-01') & (nc_emp['Dcompra_nvo'] < '2022-10-01')]
nc_mes = nc_emp.copy()
nc_mes['key'] = nc_mes['Local_creacion'].astype('str') + nc_mes['Nterminal_nvo'].astype('str') + nc_mes['Nsecuencia_nvo'].astype('str') + nc_mes['Dcompra_nvo'].apply(lambda x :x.strftime('%y%m%d'))
nc_mes.loc[nc_mes['Cautoriza'] == 10009, 'cm' ] = "Nc's código maestro"
nc_mes.loc[nc_mes['Cautoriza'] != 10009, 'cm' ] = "No código maestro"
nc_mes = nc_mes[['Local_creacion', 'Desc_local', 'Dcompra_nvo', 'Nterminal_nvo',
       'Nsecuencia_nvo', 'Hora', 'Tipo_trx', 'Cautoriza', 'Estado', 'Usuario',
       'Cvendedor', 'Local_ant', 'Descr_local_ant', 'Dcompra_ant',
       'Nterminal_ant', 'Nsecuencia_ant', 'Cvendedor_ant', 'Cajero_apertura',
       'Linea', 'LiDescripcion', 'Slinea', 'SLiDescripcion', 'Clase',
       'CDescripcion', 'SClase', 'SCDescripcion', 'SKU', 'EAN', 'PDescripcion',
       'Cmarca', 'Tipo Producto', 'Origen', 'NIT_proveedor',
       'Nombre_proveedor', 'Codigof12', 'Cevento', 'Cevento_desc',
       'Nrutcomprador', 'Qcantidad', 'Punitario_venta', 'Punitario_nc',
       'Mventa', 'Mventa_nc', 'Mdescto', 'Ctipificacion', 'Xtipificacion',
       'NOMBRE COMPLETO', 'Cargo', 'ind_pos', 'Sucursales','key', 'cm']]
# nc_mes.to_csv(f'output/{dt}_inf_cm.csv', index=False)

In [16]:
nc_mes['Linea'] = nc_mes['Linea'].str.strip()

In [21]:
nc_mes.loc[nc_mes['Mventa_nc'] >= 500000, 'ind_j11_mq'] = 'Mayor a $500.000'
nc_mes.loc[nc_mes['Mventa_nc'] < 500000, 'ind_j11_mq'] = 'Menor a $500.000'
nc_mes.to_csv(f'C:/Users/maperezr/Falabella/Panel Cambios y devoluciones - General/inf_nc.csv', index=False) #guardar inf_nc

In [18]:
print('costo: ${:,.0f} M '.format(nc_mes['Mventa_nc'].sum()/1e6))
print('cantidad: ${:,.0f}'.format(nc_mes['key'].nunique()))

costo: $170,115 M 
cantidad: $579,261


In [19]:
nc_mes.loc[nc_mes['Dcompra_nvo'] >= '2022-11-01'].to_excel('221122_nc_noviembre.xlsx')

In [20]:
asdasdasd

NameError: name 'asdasdasd' is not defined

Finaliza NC's totales


Inicia Ventas

In [ ]:
nc_mes = nc_emp.loc[(nc_emp['Dcompra_nvo'] >= '2022-10-01') & (nc_emp['Dcompra_nvo'] < '2022-11-01')]

: 

In [ ]:
ventas_mes = ventas.loc[(ventas['Día'] >= '2022-10-01') & (ventas['Día'] < '2022-11-01')]

: 

In [ ]:
gb_ventas_mes = ventas_mes.groupby(['Número de Vendedor (Cod.)'])['Venta en $'].sum().reset_index()

: 

In [ ]:
gb_nc_mes = nc_mes.groupby(['Cvendedor','Desc_local'])['Mventa_nc'].sum().reset_index()

: 

In [ ]:
m_gbs = gb_ventas_mes.merge(gb_nc_mes, how='left', left_on = 'Número de Vendedor (Cod.)',right_on = 'Cvendedor' )

: 

In [ ]:
m_gbs['dif'] = m_gbs['Venta en $'] - m_gbs['Mventa_nc']

: 

In [ ]:
com_vta_nc  = m_gbs.loc[m_gbs['dif']< 0]

: 

In [ ]:
res_empl = com_vta_nc.merge(planta[['Codigo Empleado', 'NOMBRE COMPLETO','Cargo', 'Sucursales' ]], how= 'left', left_on = 'Cvendedor', right_on = 'Codigo Empleado')
res_empl = res_empl.merge(empleados, how='left', left_on = 'Cvendedor', right_on = 'Número Vendedor')
res_empl.loc[(res_empl['NOMBRE COMPLETO'].isna()) & (res_empl['Nombre'].notna()), 'NOMBRE COMPLETO'] = res_empl.loc[(res_empl['NOMBRE COMPLETO'].isna()) & (res_empl['Nombre'].notna()), 'Nombre'].values

: 

In [ ]:
res_empl['dif'] = abs(res_empl['dif'].astype(int))
res_empl['Mventa_nc'] = res_empl['Mventa_nc'].astype(int)


: 

In [ ]:
res_empl = res_empl[['Número de Vendedor (Cod.)','NOMBRE COMPLETO', 'Cargo', 'Desc_local', 'Venta en $', 'Mventa_nc', 'dif']]

: 

In [ ]:
res_empl['Venta en $'] = res_empl['Venta en $'].astype(int)

: 

In [ ]:
res_empl.sort_values(by='dif', ascending= False)

: 

In [ ]:
res_empl.loc[res_empl['Número de Vendedor (Cod.)'] == '110787']

: 

In [ ]:
res_empl.to_csv(f'C:/Users/maperezr/OneDrive - Falabella/General/JIRA/GIGS/Descuentos/informes_panel_bi/res_emp.csv', index=False)

: 